In [1]:
import torch
from torch import nn, optim
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

/Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9A4710B9-0DA3-36BB-9129-645F282E64B2> /Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <ECC148AF-20FF-3EEE-BC75-4DD3E7455393> /Users/david/anaconda3/envs/myenv/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Othe

In [2]:
print(torch.__version__)

2.5.1


In [3]:
# Set the device
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load a pre-trained SBERT model
sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [4]:
# Define custom classification model with SBERT embeddings
class SBERTClassifier(nn.Module):
    def __init__(self, sbert_model):
        super(SBERTClassifier, self).__init__()
        self.sbert_model = sbert_model
        self.classifier = nn.Sequential(
            nn.Linear(self.sbert_model.get_sentence_embedding_dimension(), 1),
            nn.Sigmoid()  # Sigmoid to output probabilities between 0 and 1
        )
    
    def forward(self, input_texts):
        # Encode the input texts using SBERT
        embeddings = self.sbert_model.encode(input_texts, convert_to_tensor=True)
        logits = self.classifier(embeddings)
        return logits


In [5]:
# Example Dataset class
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx], torch.tensor(self.labels[idx], dtype=torch.float32)

In [6]:
# Define sample data (replace with your actual dataset)
train_texts = ["I love programming", "Python is great", "I dislike AI", "Coding is fun"]
train_labels = [1, 1, 0, 1]  # Positive class
val_texts = ["I hate bugs", "Debugging is frustrating", "I like eating"]
val_labels = [0, 0, 1]  # Negative class
test_texts = ["Learning AI is amazing", "Errors are annoying"]
test_labels = [1, 0]

# Create datasets and data loaders
train_dataset = TextDataset(train_texts, train_labels)
val_dataset = TextDataset(val_texts, val_labels)
test_dataset = TextDataset(test_texts, test_labels)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2)
test_loader = DataLoader(test_dataset, batch_size=2)

# Initialize model, criterion, and optimizer
model = SBERTClassifier(sbert_model).to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=1e-5)


In [ ]:
model(train_texts).squeeze().cpu().detach().numpy()

In [ ]:
for texts, labels in train_loader:
    print(texts)
    print(labels)

In [19]:
import numpy as np
# Training function with AUC calculation on validation set
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for texts, labels in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
            texts = [text for text in texts]
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(texts).squeeze()  # Remove extra dimension
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")
        
        # Validate after each epoch
        auc = evaluate_model(model, val_loader)
        print(f"Validation AUC after Epoch {epoch + 1}: {auc:.4f}")

# Evaluation function for AUC
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    with torch.no_grad():
        for texts, labels in data_loader:
            texts = [text for text in texts]
            labels = labels.to(device)
            
            # Get predicted probabilities
            outputs = model(texts).squeeze()  # Remove extra dimension
            all_labels.extend(labels.cpu().detach().numpy())

            outputs = outputs.cpu().detach().numpy()
            if outputs.ndim == 0:
                outputs = outputs.reshape(1)

            print(outputs)
            all_probs.extend(outputs)
    # Calculate AUC
    auc = roc_auc_score(all_labels, all_probs)
    model.train()  # Set model back to training mode
    return auc

In [ ]:
# Test function to calculate AUC on the test set
def test_model(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    with torch.no_grad():
        for texts, labels in data_loader:
            texts = [text for text in texts]
            labels = labels.to(device)
            
            outputs = model(texts).squeeze()
            all_labels.extend(labels.cpu().detach().numpy())
            all_probs.extend(outputs.cpu().detach().numpy())
    
    # Calculate AUC
    auc = roc_auc_score(all_labels, all_probs)
    print(f"Test AUC: {auc:.4f}")

In [20]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Training Epoch 1: 100%|██████████| 2/2 [00:00<00:00,  6.67it/s]


Epoch [1/10], Loss: 0.7047
[0.4892356 0.4817939]
[0.49916002]
Validation AUC after Epoch 1: 1.0000


Training Epoch 2: 100%|██████████| 2/2 [00:00<00:00, 49.28it/s]


Epoch [2/10], Loss: 0.7047
[0.48924395 0.48181123]
[0.49917406]
Validation AUC after Epoch 2: 1.0000


Training Epoch 3: 100%|██████████| 2/2 [00:00<00:00, 27.10it/s]


...

Epoch [8/10], Loss: 0.7042
[0.4893012 0.4819178]
[0.49926472]
Validation AUC after Epoch 8: 1.0000


Training Epoch 9: 100%|██████████| 2/2 [00:00<00:00, 51.97it/s]


Epoch [9/10], Loss: 0.7042
[0.4893101 0.4819353]
[0.49927923]
Validation AUC after Epoch 9: 1.0000


Training Epoch 10: 100%|██████████| 2/2 [00:00<00:00, 50.24it/s]


Epoch [10/10], Loss: 0.7041
[0.489321   0.48195368]
[0.49929544]
Validation AUC after Epoch 10: 1.0000
